# S&P 500 Stochastic Volatility Forecasting

## Quick Demo: Stochastic Models + Machine Learning

This notebook demonstrates:
1. **GARCH(1,1)** - Classical stochastic volatility model
2. **ML Ensemble** - LightGBM, Random Forest, Ridge
3. **Comparison** - Stochastic baseline vs ML predictions

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_loader import SP500DataLoader
from stochastic_models import GARCHModel, SimpleStochasticVol
from feature_engineering import VolatilityFeatureEngineer, create_train_test_split
from ml_models import VolatilityMLEnsemble

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Imports successful")

## 1. Load S&P 500 Data

In [ ]:
# Load prepared data
loader = SP500DataLoader()
df = loader.load_cached_data("sp500_prepared.csv")

print(f"Dataset: {len(df)} samples")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
print(f"\nColumns: {df.columns.tolist()[:10]}...")

# Plot S&P 500 price and realized volatility
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(df.index, df['close'], linewidth=1.5)
axes[0].set_title('S&P 500 Price', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Price')
axes[0].grid(True, alpha=0.3)

axes[1].plot(df.index, df['realized_vol_20'], color='red', linewidth=1.5)
axes[1].set_title('Realized Volatility (20-period)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Volatility')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Stochastic Models: GARCH(1,1)

GARCH captures **volatility clustering**:
$$\sigma^2_t = \omega + \alpha \cdot r^2_{t-1} + \beta \cdot \sigma^2_{t-1}$$

Where:
- $\alpha$ = impact of recent shocks (news)
- $\beta$ = persistence of volatility
- $\alpha + \beta$ close to 1 = high persistence

In [ ]:
# Fit GARCH model
garch = GARCHModel(p=1, q=1)
garch.fit(df['returns'])

# Get conditional volatility
cond_vol = garch.get_conditional_volatility()

# Plot
plt.figure(figsize=(14, 6))
plt.plot(df.index[-len(cond_vol):], cond_vol, label='GARCH Conditional Vol', linewidth=1.5)
plt.plot(df.index, df['realized_vol_20'], label='Realized Vol (20-period)', alpha=0.7, linewidth=1.5)
plt.title('GARCH(1,1) Conditional Volatility vs Realized Volatility', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✓ GARCH parameters:")
print(f"  α (alpha) = {garch.alpha:.4f} - news impact")
print(f"  β (beta)  = {garch.beta:.4f} - persistence")
print(f"  α + β     = {garch.alpha + garch.beta:.4f}")
print(f"\nInterpretation: {garch.alpha + garch.beta:.2%} of volatility persists to next period")

## 3. Feature Engineering for ML

In [ ]:
# Create features
engineer = VolatilityFeatureEngineer()
df_features = engineer.create_all_features(df)

print(f"Original features: {len(df.columns)}")
print(f"After engineering: {len(df_features.columns)}")
print(f"New features created: {len(engineer.get_feature_names())}")

# Show some feature examples
print("\nExample features:")
for i, feat in enumerate(engineer.get_feature_names()[:15], 1):
    print(f"  {i:2d}. {feat}")

## 4. Train ML Ensemble

In [ ]:
# Split data
X_train, X_test, y_train, y_test = create_train_test_split(df_features, test_size=0.2)

# Further split for validation
val_size = int(len(X_train) * 0.2)
X_val = X_train.iloc[-val_size:]
y_val = y_train.iloc[-val_size:]
X_train = X_train.iloc[:-val_size]
y_train = y_train.iloc[:-val_size]

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

In [ ]:
# Train ensemble
ensemble = VolatilityMLEnsemble()
ensemble.fit(X_train, y_train, X_val, y_val)

## 5. Evaluate on Test Set

In [ ]:
# Evaluate
results = ensemble.evaluate(X_test, y_test)

# Get predictions
ensemble_pred = ensemble.predict(X_test)

In [ ]:
# Plot predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Time series
ax = axes[0]
ax.plot(y_test.index, y_test.values, label='Actual', alpha=0.7, linewidth=1.5)
ax.plot(y_test.index, ensemble_pred, label='ML Ensemble', alpha=0.7, linewidth=1.5)
ax.set_title('ML Ensemble: Predicted vs Actual Volatility', fontsize=14, fontweight='bold')
ax.set_ylabel('Volatility')
ax.legend()
ax.grid(True, alpha=0.3)

# Scatter
ax = axes[1]
ax.scatter(y_test, ensemble_pred, alpha=0.5, s=20)
min_val, max_val = y_test.min(), y_test.max()
ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect', linewidth=2)
ax.set_xlabel('Actual Volatility')
ax.set_ylabel('Predicted Volatility')
ax.set_title('Predicted vs Actual (Scatter)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Feature Importance

In [ ]:
# Get feature importance
importance_df = ensemble.get_feature_importance(X_train.columns, top_n=20)

# Plot
plt.figure(figsize=(10, 8))
top_features = importance_df.head(20).sort_values('importance')
plt.barh(range(len(top_features)), top_features['importance'].values)
plt.yticks(range(len(top_features)), top_features['feature'].values)
plt.xlabel('Importance', fontsize=12)
plt.title('Top 20 Most Important Features', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
print(importance_df.head(10).to_string(index=False))

## 7. Summary

### Key Takeaways:

1. **Stochastic Models (GARCH)**:
   - Captures volatility clustering
   - Mean-reverting behavior
   - Provides interpretable baseline

2. **Machine Learning Ensemble**:
   - Learns complex non-linear patterns
   - Combines multiple models (LightGBM, RF, Ridge)
   - Outperforms pure stochastic approaches

3. **Applications**:
   - **Risk Management**: Position sizing, VaR estimation
   - **Options Trading**: Implied vs realized vol arbitrage
   - **Portfolio Optimization**: Dynamic allocation based on vol regime

In [ ]:
# Final metrics summary
print("="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"\nEnsemble Performance:")
print(f"  MAE:  {results['ensemble']['mae']:.6f}")
print(f"  RMSE: {results['ensemble']['rmse']:.6f}")
print(f"  R²:   {results['ensemble']['r2']:.4f}")
print(f"\nImprovement over baseline: {results['improvement_pct']:.1f}%")
print("\n✓ Project complete!")